# Scraping Huizenzoeker.nl to Analyse the Dutch Housing Market

### Introduction
Which places in the Netherlands are hit hardest by the Dutch Housing crisis, and which the least?
Momentarily, the housing crisis is one of the most prominent societal challenges in the Netherlands. This script scrapes information of the Dutch housing market, enabling use to analyse the housing market and clearify which areas are hit hardest by the housing crisis. This script provides information such as gem. vraagprijs, # verkochte woningen, gem. vierkante meter prijs, and % overboden. The dataframe that is generated through this script offers very interesting information, for example for first-time buyers that are having a hard time purchasing their first home on the current stressed Dutch housing market.

The script is devided into seven steps:
* **Step 1. Loading all the basics**: this step loads all the relevent packages and sets up the BeautifulSoup basis.
* **Step 2. Collecting the municipality URLs**: this step collects the urls of the municipalities in the Netherlands. For this step, we first need to create a list of the province URLs (twelve in total; for each province in the Netherlands). From these twelve province URLs, we are able to scrape the municipality URLs, since each province URL offers the opportunity to navigate to their corresponding municipalities.
* **Step 3. Scraping data from URLs (municipality-level)**: this step scrapes the data from the municipality URLs that we have generated in step 2.
* **Step 4: Exporting dataframe as CSV file**: this step exports the final dataframe as CSV file.
* **Step 5: Providing summary statistics**: this step provides summary statistics for the final dataframe that we have generated in R, by loading the CSV file that we exported in step 4 in R.
* **Step 6: Scraping data from woningmarkt dashboard (province-level)**: this step scrapes the data from each province. The same code that was used to scraping on municipality-level is employed.
* **Step 7: Exporting dashboard data as CSV file**: this step exports the provinces data as CSV file.

## Step 1: Loading all the basics

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd 
import time 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import json

In [2]:
url = 'https://www.huizenzoeker.nl/woningmarkt/'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

## Step 2: Collecting the municipality URLs

We first construct a base url and a province_url which once appended together creates the URL to each of the woningmarkt pages for each province. We generate the generate_links() function to append these parts of the URL together. 

In [3]:
base_url = 'https://www.huizenzoeker.nl/woningmarkt/'
province_url = ['noord-holland/', 'zuid-holland/', 'zeeland/', 'noord-brabant/', 'utrecht/', 'flevoland/', 
                'friesland/', 'groningen/', 'drenthe/', 'overijssel/', 'gelderland/', 'limburg/']

In [4]:
def generate_links(base_url,province_url): 
    page_links = []
    for i in province_url:
        full_links = base_url + i
        page_links.append(full_links)  
    return page_links

page_links = generate_links(base_url,province_url)

We then use this list of all provinces, to extract all municipalities from each, making use of window handling. 

In [5]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/lesleyhaerkens/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


In [6]:
page_urls_full = []

for link in page_links:
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)
    # time.sleep(2)
    
    for elem in driver.find_elements_by_xpath("//li//div//a[@href]"):
        urls = elem.get_attribute('href')
        page_urls_full.append(urls)

In [8]:
subset = page_urls_full[30:33] # use subset to try out on few urls (for time convenience)

## Step 3: Scrape data from each url (municipality-level)

For each municipality we extract:
* *trend data*: gem. vraagprijs, verkochte woningen, gem.vierkantemeter prijs, % overboden (and how these numbers how changed t.o.v. vorige maand) 
* *other information*: besteedbaar inkomen, aantal inwoners

#### Warning: Running the next cell for 'page_urls_full' will take aprox. 30 minutes. You might want to replace page_urls_full for 'subset'!

In [9]:
fn = 'saved_data.json'

def extract_city_trends(subset):
    trend_list = []
    for page_url in subset:
        driver.get(page_url)
        time.sleep(5) 
        soup = BeautifulSoup(driver.page_source, 'html.parser')
            # City name
        city_name = soup.find_all('h2')[0].get_text()
        city_name = city_name.replace('Woningmarkt','')
        city_name = city_name.replace(' ', '')
            # Gemiddelde vraagprijs
        content = soup.find_all(class_='trend-graph')[0]
        if content.find(class_="trend-graph-icon") == None:
            gem_vraagprijs = content.find("h3",{"class":"trend-graph-value"}).get_text()
            tov_vorige_maand_vraagprijs = "NA"
        else:
            if content.find(class_="trend-graph-pill trend-down") != None:
                gem_vraagprijs = content.find("h3",{"class":"trend-graph-value"}).get_text()
                gem_vraagprijs = gem_vraagprijs.replace("(","")
                gem_vraagprijs = gem_vraagprijs.replace(",)","")
                gem_vraagprijs = gem_vraagprijs.replace(".", ",")
                tov_vorige_maand_vraagprijs = content.find("div",{"class":"trend-graph-pill trend-down"}).get_text()
                tov_vorige_maand_vraagprijs = tov_vorige_maand_vraagprijs.replace("\n\n","")
                tov_vorige_maand_vraagprijs = tov_vorige_maand_vraagprijs.replace(" t.o.v. vorige maand\n","")
            else:
                gem_vraagprijs = content.find("h3",{"class":"trend-graph-value"}).get_text()
                gem_vraagprijs = gem_vraagprijs.replace("(","")
                gem_vraagprijs = gem_vraagprijs.replace(",)","")    
                gem_vraagprijs = gem_vraagprijs.replace(".", ",")
                tov_vorige_maand_vraagprijs = content.find("div",{"class":"trend-graph-pill"}).get_text()
                tov_vorige_maand_vraagprijs = tov_vorige_maand_vraagprijs.replace("\n\n","")
                tov_vorige_maand_vraagprijs = tov_vorige_maand_vraagprijs.replace(" t.o.v. vorige maand\n","")
            # Aantal verkochte woningen
        content = soup.find_all(class_='trend-graph')[1]
        if content.find(class_="trend-graph-icon") == None:
            verk_woningen = content.find("h3",{"class":"trend-graph-value"}).get_text()
            tov_vorige_maand_verkocht = "NA"
        else:
            if content.find(class_="trend-graph-pill trend-down") != None:
                verk_woningen = content.find("h3",{"class":"trend-graph-value"}).get_text()               
                tov_vorige_maand_verkocht = content.find("div",{"class":"trend-graph-pill trend-down"}).get_text()
                tov_vorige_maand_verkocht = tov_vorige_maand_verkocht.replace("\n\n","")
                tov_vorige_maand_verkocht = tov_vorige_maand_verkocht.replace(" t.o.v. vorige maand\n","")
            else:
                verk_woningen = content.find("h3",{"class":"trend-graph-value"}).get_text()             
                tov_vorige_maand_verkocht = content.find("div",{"class":"trend-graph-pill"}).get_text()
                tov_vorige_maand_verkocht = tov_vorige_maand_verkocht.replace("\n\n","")
                tov_vorige_maand_verkocht = tov_vorige_maand_verkocht.replace(" t.o.v. vorige maand\n","")
            # Gemiddelde vierkante meter prijs
        content = soup.find_all(class_='trend-graph')[2]
        if content.find(class_="trend-graph-icon") == None:
            m2_prijs = content.find("h3",{"class":"trend-graph-value"}).get_text()
            tov_vorige_maand_m2_prijs = "NA"
        else:
            if content.find(class_="trend-graph-pill trend-down") != None:
                m2_prijs = content.find("h3",{"class":"trend-graph-value"}).get_text()     
                m2_prijs = m2_prijs.replace(".", ",")
                tov_vorige_maand_m2_prijs = content.find("div",{"class":"trend-graph-pill trend-down"}).get_text()
                tov_vorige_maand_m2_prijs = tov_vorige_maand_m2_prijs.replace("\n\n","")
                tov_vorige_maand_m2_prijs = tov_vorige_maand_m2_prijs.replace(" t.o.v. vorige maand\n","")
            else:
                m2_prijs = content.find("h3",{"class":"trend-graph-value"}).get_text() 
                m2_prijs = m2_prijs.replace(".", ",")
                tov_vorige_maand_m2_prijs = content.find("div",{"class":"trend-graph-pill"}).get_text() 
                tov_vorige_maand_m2_prijs = tov_vorige_maand_m2_prijs.replace("\n\n","")
                tov_vorige_maand_m2_prijs = tov_vorige_maand_m2_prijs.replace(" t.o.v. vorige maand\n","")
            # Percentage overboden
        content = soup.find_all(class_='trend-graph')[3]
        if content.find(class_="trend-graph-icon") == None:
            perc_overboden = content.find("h3",{"class":"trend-graph-value"}).get_text()
            tov_vorige_maand_perc_overboden = "NA"
        else:
            if content.find(class_="trend-graph-pill trend-down") != None:
                perc_overboden = content.find("h3",{"class":"trend-graph-value"}).get_text()               
                tov_vorige_maand_perc_overboden = content.find("div",{"class":"trend-graph-pill trend-down"}).get_text()
                tov_vorige_maand_perc_overboden = tov_vorige_maand_perc_overboden.replace("\n\n","")
                tov_vorige_maand_perc_overboden = tov_vorige_maand_perc_overboden.replace(" t.o.v. vorige maand\n","")
            else:
                perc_overboden = content.find("h3",{"class":"trend-graph-value"}).get_text()             
                tov_vorige_maand_perc_overboden = content.find("div",{"class":"trend-graph-pill"}).get_text()
                tov_vorige_maand_perc_overboden = tov_vorige_maand_perc_overboden.replace("\n\n","")
                tov_vorige_maand_perc_overboden = tov_vorige_maand_perc_overboden.replace(" t.o.v. vorige maand\n","")
            # Besteedbaar inkomen
        bes_inkomen = soup.find_all(class_='detail__income huizenzoeker-card single-value-graph-container')[0].get_text()
        bes_inkomen = bes_inkomen.replace('\n','')
        bes_inkomen = bes_inkomen.replace('Besteedbaar Inkomen Per Huishouden','')
        bes_inkomen = bes_inkomen.replace(".", ",")
            # Inwoners
        content = soup.find("div", {"class": "buurt-info"})
        inwoners = content.find_all('p')[3].get_text
        inwoners = str(inwoners)
        inwoners = re.search('Dat zijn(.+?)inwoners', inwoners)
        if inwoners:
            found_inwoners = inwoners.group(1)
            found_inwoners = found_inwoners.strip()
            found_inwoners = found_inwoners.replace(".", ",")
        else:
            found_inwoners = 'NA'
            # Bevolkingsgroei
        content = soup.find("div", {"class": "buurt-info"})
        populatiegroei = content('p')[4].get_text
        populatiegroei = str(populatiegroei)
        populatiegroei_increase = re.search('afgelopen jaar met (.+?) gegroeid', populatiegroei)
        if populatiegroei_increase:
            found_populatiegroei = populatiegroei_increase.group(1)
            found_populatiegroei = found_populatiegroei.strip()
        else:
            found_populatiegroei = 'NA'
        populatiegroei_decline = re.search('afgelopen jaar met (.+?) gekrompen', populatiegroei)
        if populatiegroei_decline:
            found_populatiegroei_decline = populatiegroei_decline.group(1)
            found_populatiegroei_decline = found_populatiegroei_decline.strip() 
        else:
            found_populatiegroei_decline = 'NA'
            # Append list
        save_obj = {"City":city_name, 
                    "Gem. vraagprijs":gem_vraagprijs, "%Δ Vraagprijs (t.o.v vorige maand)": tov_vorige_maand_vraagprijs,
                    "Verkochte woningen":verk_woningen, "%Δ Verkochte woningen (t.o.v vorige maand)":tov_vorige_maand_verkocht,
                    "Gem. m2 prijs":m2_prijs, "%Δ M2 prijs (t.o.v vorige maand)":tov_vorige_maand_m2_prijs,
                    "% Vraagprijs overboden":perc_overboden, "%Δ Overboden (t.o.v vorige maand)":tov_vorige_maand_perc_overboden,
                    "Besteedbaar inkomen (per huishouden)":bes_inkomen,
                    "Aantal inwoners": found_inwoners,
                    "% Populatie stijging":found_populatiegroei, "% Populatie daling":found_populatiegroei_decline}
        trend_list.append(save_obj)
        f=open(fn, 'a', encoding='utf-8')
        f.write(json.dumps(save_obj)+'\n')
        f.close()
    return(trend_list)

In [10]:
df = extract_city_trends(subset) 
pd.DataFrame(df)

,City,Gem. vraagprijs,%Δ Vraagprijs (t.o.v vorige maand),Verkochte woningen,%Δ Verkochte woningen (t.o.v vorige maand),Gem. m2 prijs,%Δ M2 prijs (t.o.v vorige maand),% Vraagprijs overboden,%Δ Overboden (t.o.v vorige maand),Besteedbaar inkomen (per huishouden),Aantal inwoners,% Populatie stijging,% Populatie daling
0,Medemblik,"€ 387,000",0.78%,10,-60.00%,"€ 3,131",-7.80%,14.97%,4.30%,"€ 40,500","45,101",0.65%,NA
1,Oostzaan,"€ 587,500",48.73%,2,-66.67%,"€ 4,478",18.81%,7.78%,-5.15%,"€ 46,000","9,735",NA,0.23%
2,Opmeer,"€ 642,500",133.64%,2,100.00%,"€ 3,006",20.24%,14.55%,0.21%,"€ 41,700","11,836",0.48%,NA


In [27]:
# df = extract_city_trends(page_urls_full) 
# pd.DataFrame(df)

,City,Gem. vraagprijs,%Δ Vraagprijs (t.o.v vorige maand),Verkochte woningen,%Δ Verkochte woningen (t.o.v vorige maand),Gem. m2 prijs,%Δ M2 prijs (t.o.v vorige maand),% Vraagprijs overboden,%Δ Overboden (t.o.v vorige maand),Besteedbaar inkomen (per huishouden),Aantal inwoners,% Populatie stijging,% Populatie daling
0,Aalsmeer,"€ 685,000",57.47%,12,-7.69%,"€ 4,476",9.22%,10.67%,3.42%,"€ 45,800","31,859",0.41%,NA
1,Alkmaar,"€ 372,500",28.45%,34,-46.03%,"€ 3,970",11.86%,14.05%,1.43%,"€ 36,300","109,436",0.81%,NA
2,Amstelveen,"€ 570,000",18.13%,21,-56.25%,"€ 4,724",1.88%,305.01%,296.30%,"€ 37,800","91,675",0.92%,NA
3,Amsterdam,"€ 465,000",10.71%,216,-30.99%,"€ 6,942",5.52%,16.10%,0.38%,"€ 30,100","872,757",1.13%,NA
4,Beemster,"€ 612,000",-12.26%,4,-33.33%,"€ 4,311",-6.89%,-0.23%,-12.10%,"€ 47,300","10,022",2.81%,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,ValkenburgaandeGeul,"€ 365,000",-5.19%,4,-55.56%,"€ 3,308",9.75%,12.60%,5.41%,"€ 35,600","16,367",NA,0.63%
348,Venlo,"€ 334,500",15.74%,31,-49.18%,"€ 2,735",17.94%,7.84%,-0.72%,"€ 33,700","101,802",0.20%,NA
349,Venray,"€ 297,000",8.99%,8,-33.33%,"€ 2,729",23.09%,8.98%,1.22%,"€ 39,100","43,614",0.66%,NA
350,Voerendaal,"€ 287,500",-11.13%,2,-75.00%,"€ 2,185",-12.53%,7.69%,-2.09%,"€ 40,800","12,475",0.18%,NA


In [12]:
final_dataframe=pd.DataFrame(df) #dataframe with all data for all municipalities in the Netherlands

## Step 4: Exporting dataframe as CSV file

For the final output (a CSV file, so tabular data) we would want the output of the scraper to be gathered in one single dictionary.

In [13]:
final_dataframe.to_csv('huizenzoeker_scraper_data.csv') 

## Step 5: Providing summary statistics

**Exporting our output to RStudio and then importing that CSV here**

First we try to generate some summary statistics by using the output of our scraping data. We can't do this directly as you can see as most variables are seen as characters, while they should be numerics. Therefore we exported the final_dataframe to R to change these datatypes and then export it as CSV to then use it here to generate some summary statistics: count, mean, std, min, max, 25%, 50%, 75%. 

In [14]:
huizenzoeker = pd.read_csv('huizenzoeker_data2.csv', encoding= 'latin-1') #Adding the latin encoding solved the UNICODE error 

In [15]:
huizenzoeker = pd.DataFrame(huizenzoeker)
huizenzoeker

,City,gem_vraagprijs,perc_ver_vraagprijs,verk_woningen,perc_ver_verkocht,gem_m2prijs,perc_ver_m2prijs,perc_overboden,perc_ver_overboden,best_inkomen
0,Aalsmeer,725000.0,55.41,9,-25.00,4297.0,3.19,7.25,-3.13,45800.0
1,Alkmaar,410000.0,43.86,24,-59.32,4013.0,13.94,12.67,1.53,36300.0
2,Amstelveen,700000.0,47.37,14,-68.89,5097.0,10.09,8.71,0.81,37800.0
3,Amsterdam,465000.0,9.41,166,-43.73,6993.0,5.52,15.72,1.81,30100.0
4,Beemster,675000.0,-3.23,3,-50.00,4299.0,-7.15,11.87,1.79,47300.0
...,...,...,...,...,...,...,...,...,...,...
347,ValkenburgaandeGeul,325000.0,-15.58,3,-66.67,2567.0,-14.83,7.19,10.08,35600.0
348,Venlo,379500.0,43.21,26,-50.00,2735.0,17.84,8.56,3.05,33700.0
349,Venray,297000.0,14.23,8,-27.27,2729.0,27.11,7.76,-1.93,39100.0
350,Voerendaal,287500.0,3.23,2,-66.67,2185.0,-12.53,9.78,-0.59,40800.0


In [16]:
huizenzoeker.describe() #to return the summary statisitics for all municipalities in the Netherlands. 

,gem_vraagprijs,perc_ver_vraagprijs,verk_woningen,perc_ver_verkocht,gem_m2prijs,perc_ver_m2prijs,perc_overboden,perc_ver_overboden,best_inkomen
count,3.520000e+02,339.000000,352.000000,345.000000,352.000000,339.000000,348.000000,346.000000,349.000000
mean,3.754648e+05,12.085398,13.428977,-35.358841,3157.700009,5.462271,9.443190,1.003844,40037.535817
std,1.722118e+05,32.366931,20.007992,47.145009,1078.111676,17.085127,3.842578,3.379959,4728.094766
min,0.000000e+00,-40.860000,0.000000,-100.000000,0.000000,-31.750000,-0.940000,-9.500000,25400.000000
25%,2.950000e+05,-8.470000,5.000000,-61.900000,2612.000000,-4.065000,7.050000,-0.785000,36900.000000
50%,3.500000e+05,7.690000,8.000000,-46.150000,3070.000000,4.230000,8.980000,0.835000,40400.000000
75%,4.262000e+05,25.035000,14.000000,-22.220000,3653.750000,12.695000,12.240000,2.615000,43500.000000
max,1.847500e+06,197.060000,166.000000,250.000000,9262.000000,122.210000,22.320000,19.860000,53800.000000


## Step 6: Scraping woningmarkt dashboard (province-level)

### Step 6a: First scraping the trend, and other data

Generating links of all provinces: 

In [17]:
base_url = 'https://www.huizenzoeker.nl/woningmarkt/'
province_url = ['noord-holland/', 'zuid-holland/', 'zeeland/', 'noord-brabant/', 'utrecht/', 'flevoland/', 
                'friesland/', 'groningen/', 'drenthe/', 'overijssel/', 'gelderland/', 'limburg/']

Defining a function to paste together these URL parts: 

In [18]:
def generate_links(base_url,province_url): 
    page_links = []
    for i in province_url:
        full_links = base_url + i
        page_links.append(full_links)  
    return page_links
page_links = generate_links(base_url,province_url)
print(page_links)

['https://www.huizenzoeker.nl/woningmarkt/noord-holland/', 'https://www.huizenzoeker.nl/woningmarkt/zuid-holland/', 'https://www.huizenzoeker.nl/woningmarkt/zeeland/', 'https://www.huizenzoeker.nl/woningmarkt/noord-brabant/', 'https://www.huizenzoeker.nl/woningmarkt/utrecht/', 'https://www.huizenzoeker.nl/woningmarkt/flevoland/', 'https://www.huizenzoeker.nl/woningmarkt/friesland/', 'https://www.huizenzoeker.nl/woningmarkt/groningen/', 'https://www.huizenzoeker.nl/woningmarkt/drenthe/', 'https://www.huizenzoeker.nl/woningmarkt/overijssel/', 'https://www.huizenzoeker.nl/woningmarkt/gelderland/', 'https://www.huizenzoeker.nl/woningmarkt/limburg/']


In [19]:
def extract_province_trends(page_links):
    trend_list = []
    for page_link in page_links:
        driver.get(page_link)
        time.sleep(1) 
        soup = BeautifulSoup(driver.page_source, 'html.parser')
            # City name
        province_name = soup.find_all('h2')[0].get_text()
        province_name = province_name.replace('Woningmarkt','')
        province_name = province_name.replace(' ', '')
            # Gemiddelde vraagprijs
        content = soup.find_all(class_='trend-graph')[0]
        if content.find(class_="trend-graph-icon") == None:
            gem_vraagprijs = content.find("h3",{"class":"trend-graph-value"}).get_text()
            tov_vorige_maand_vraagprijs = "NA"
        else:
            if content.find(class_="trend-graph-pill trend-down") != None:
                gem_vraagprijs = content.find("h3",{"class":"trend-graph-value"}).get_text()
                gem_vraagprijs = gem_vraagprijs.replace("(","")
                gem_vraagprijs = gem_vraagprijs.replace(",)","")
                gem_vraagprijs = gem_vraagprijs.replace(".", ",")
                tov_vorige_maand_vraagprijs = content.find("div",{"class":"trend-graph-pill trend-down"}).get_text()
                tov_vorige_maand_vraagprijs = tov_vorige_maand_vraagprijs.replace("\n\n","")
                tov_vorige_maand_vraagprijs = tov_vorige_maand_vraagprijs.replace(" t.o.v. vorige maand\n","")
            else:
                gem_vraagprijs = content.find("h3",{"class":"trend-graph-value"}).get_text()
                gem_vraagprijs = gem_vraagprijs.replace("(","")
                gem_vraagprijs = gem_vraagprijs.replace(",)","")    
                gem_vraagprijs = gem_vraagprijs.replace(".", ",")
                tov_vorige_maand_vraagprijs = content.find("div",{"class":"trend-graph-pill"}).get_text()
                tov_vorige_maand_vraagprijs = tov_vorige_maand_vraagprijs.replace("\n\n","")
                tov_vorige_maand_vraagprijs = tov_vorige_maand_vraagprijs.replace(" t.o.v. vorige maand\n","")
            # Aantal verkochte woningen
        content = soup.find_all(class_='trend-graph')[1]
        if content.find(class_="trend-graph-icon") == None:
            verk_woningen = content.find("h3",{"class":"trend-graph-value"}).get_text()
            tov_vorige_maand_verkocht = "NA"
        else:
            if content.find(class_="trend-graph-pill trend-down") != None:
                verk_woningen = content.find("h3",{"class":"trend-graph-value"}).get_text()               
                tov_vorige_maand_verkocht = content.find("div",{"class":"trend-graph-pill trend-down"}).get_text()
                tov_vorige_maand_verkocht = tov_vorige_maand_verkocht.replace("\n\n","")
                tov_vorige_maand_verkocht = tov_vorige_maand_verkocht.replace(" t.o.v. vorige maand\n","")
            else:
                verk_woningen = content.find("h3",{"class":"trend-graph-value"}).get_text()             
                tov_vorige_maand_verkocht = content.find("div",{"class":"trend-graph-pill"}).get_text()
                tov_vorige_maand_verkocht = tov_vorige_maand_verkocht.replace("\n\n","")
                tov_vorige_maand_verkocht = tov_vorige_maand_verkocht.replace(" t.o.v. vorige maand\n","")
            # Gemiddelde vierkante meter prijs
        content = soup.find_all(class_='trend-graph')[2]
        if content.find(class_="trend-graph-icon") == None:
            m2_prijs = content.find("h3",{"class":"trend-graph-value"}).get_text()
            tov_vorige_maand_m2_prijs = "NA"
        else:
            if content.find(class_="trend-graph-pill trend-down") != None:
                m2_prijs = content.find("h3",{"class":"trend-graph-value"}).get_text()     
                m2_prijs = m2_prijs.replace(".", ",")
                tov_vorige_maand_m2_prijs = content.find("div",{"class":"trend-graph-pill trend-down"}).get_text()
                tov_vorige_maand_m2_prijs = tov_vorige_maand_m2_prijs.replace("\n\n","")
                tov_vorige_maand_m2_prijs = tov_vorige_maand_m2_prijs.replace(" t.o.v. vorige maand\n","")
            else:
                m2_prijs = content.find("h3",{"class":"trend-graph-value"}).get_text() 
                m2_prijs = m2_prijs.replace(".", ",")
                tov_vorige_maand_m2_prijs = content.find("div",{"class":"trend-graph-pill"}).get_text() 
                tov_vorige_maand_m2_prijs = tov_vorige_maand_m2_prijs.replace("\n\n","")
                tov_vorige_maand_m2_prijs = tov_vorige_maand_m2_prijs.replace(" t.o.v. vorige maand\n","")
            # Percentage overboden
        content = soup.find_all(class_='trend-graph')[3]
        if content.find(class_="trend-graph-icon") == None:
            perc_overboden = content.find("h3",{"class":"trend-graph-value"}).get_text()
            tov_vorige_maand_perc_overboden = "NA"
        else:
            if content.find(class_="trend-graph-pill trend-down") != None:
                perc_overboden = content.find("h3",{"class":"trend-graph-value"}).get_text()               
                tov_vorige_maand_perc_overboden = content.find("div",{"class":"trend-graph-pill trend-down"}).get_text()
                tov_vorige_maand_perc_overboden = tov_vorige_maand_perc_overboden.replace("\n\n","")
                tov_vorige_maand_perc_overboden = tov_vorige_maand_perc_overboden.replace(" t.o.v. vorige maand\n","")
            else:
                perc_overboden = content.find("h3",{"class":"trend-graph-value"}).get_text()             
                tov_vorige_maand_perc_overboden = content.find("div",{"class":"trend-graph-pill"}).get_text()
                tov_vorige_maand_perc_overboden = tov_vorige_maand_perc_overboden.replace("\n\n","")
                tov_vorige_maand_perc_overboden = tov_vorige_maand_perc_overboden.replace(" t.o.v. vorige maand\n","")
            # Besteedbaar inkomen
        bes_inkomen = soup.find_all(class_='detail__income huizenzoeker-card single-value-graph-container')[0].get_text()
        bes_inkomen = bes_inkomen.replace('\n','')
        bes_inkomen = bes_inkomen.replace('Besteedbaar Inkomen Per Huishouden','')
        bes_inkomen = bes_inkomen.replace(".", ",")
            # Inwoners
        content = soup.find("div", {"class": "buurt-info"})
        inwoners = content.find_all('p')[3].get_text
        inwoners = str(inwoners)
        inwoners = re.search('Dat zijn(.+?)inwoners', inwoners)
        found_inwoners = 'NA'
        if inwoners:
            found_inwoners = inwoners.group(1)
            found_inwoners = found_inwoners.strip()
            found_inwoners = found_inwoners.replace(".", ",")
            # Bevolkingsgroei
        content = soup.find("div", {"class": "buurt-info"})
        populatiegroei = content('p')[4].get_text
        populatiegroei = str(populatiegroei)
        populatiegroei_increase = re.search('afgelopen jaar met (.+?) gegroeid', populatiegroei)
        if populatiegroei_increase:
            found_populatiegroei = populatiegroei_increase.group(1)
            found_populatiegroei = found_populatiegroei.strip()
        else:
            found_populatiegroei = 'NA'
        populatiegroei_decline = re.search('afgelopen jaar met (.+?) gekrompen', populatiegroei)
        if populatiegroei_decline:
            found_populatiegroei_decline = populatiegroei_decline.group(1)
            found_populatiegroei_decline = found_populatiegroei_decline.strip() 
        else:
            found_populatiegroei_decline = 'NA'
            # Append list
        trend_list.append({"Province":province_name, 
                    "Gem. vraagprijs":gem_vraagprijs, "%Δ Vraagprijs (t.o.v vorige maand)": tov_vorige_maand_vraagprijs,
                    "Verkochte woningen":verk_woningen, "%Δ Verkochte woningen (t.o.v vorige maand)":tov_vorige_maand_verkocht,
                    "Gem. m2 prijs":m2_prijs, "%Δ M2 prijs (t.o.v vorige maand)":tov_vorige_maand_m2_prijs,
                    "% Vraagprijs overboden":perc_overboden, "%Δ Overboden (t.o.v vorige maand)":tov_vorige_maand_perc_overboden,
                    "Besteedbaar inkomen (per huishouden)":bes_inkomen,
                    "Aantal inwoners": found_inwoners,
                    "% Populatie stijging":found_populatiegroei, "% Populatie daling":found_populatiegroei_decline})
    return(trend_list)

In [20]:
df2 = extract_province_trends(page_links) 
province_dataframe = pd.DataFrame(df2)
province_dataframe

,Province,Gem. vraagprijs,%Δ Vraagprijs (t.o.v vorige maand),Verkochte woningen,%Δ Verkochte woningen (t.o.v vorige maand),Gem. m2 prijs,%Δ M2 prijs (t.o.v vorige maand),% Vraagprijs overboden,%Δ Overboden (t.o.v vorige maand),Besteedbaar inkomen (per huishouden),Aantal inwoners,% Populatie stijging,% Populatie daling
0,Noord-Holland,"€ 425,000",13.33%,936,-26.76%,"€ 4,459",10.04%,28.66%,16.00%,"€ 36,200","2,879,527",0.92%,NA
1,Zuid-Holland,"€ 365,000",7.67%,1236,-39.44%,"€ 3,618",6.07%,20.98%,10.77%,"€ 35,800","3,708,696",0.95%,NA
2,Zeeland,"€ 279,750",1.91%,161,-40.59%,"€ 2,663",4.43%,10.05%,1.92%,"€ 36,900","383,488",0.12%,NA
3,Noord-Brabant,"€ 356,750",5.24%,714,-47.11%,"€ 3,209",5.87%,8.74%,0.87%,"€ 38,100","2,548,585",0.71%,NA
4,Utrecht,"€ 430,500",11.82%,587,-10.65%,"€ 4,195",4.09%,13.03%,1.06%,"€ 39,500","1,354,834",0.94%,NA
5,Flevoland,"€ 335,000",3.08%,149,-40.40%,"€ 2,982",1.71%,16.40%,1.69%,"€ 39,500","423,021",1.55%,NA
6,Friesland,"€ 287,000",4.36%,314,-14.21%,"€ 2,432",-1.58%,11.71%,1.07%,"€ 34,900","649,957",0.35%,NA
7,Groningen,"€ 250,000",11.11%,310,-9.88%,"€ 2,564",8.37%,20.27%,4.51%,"€ 30,600","540,009",0.38%,NA
8,Drenthe,"€ 308,000",4.41%,229,-27.53%,"€ 2,477",1.27%,11.58%,1.09%,"€ 37,100","493,682",0.31%,NA
9,Overijssel,"€ 300,000",1.69%,441,-18.18%,"€ 2,693",4.46%,10.50%,0.66%,"€ 36,900","1,162,406",0.52%,NA


### Step 6b: Scrape some more woningmarkt dashboard data

To this dataframe, we now want to add more data from the woningmarkt dashboard per province, e.g. 'aantal geintereseerden per woning', huuraanbod, profiel huizenzoekers (?), over woningen...  

But for now,  we export this dataframe already as CSV to R to fix the characters into numerics; such that it is an useable dataset!

## Step 7: Exporting dashboard data as CSV

In [21]:
province_dataframe.to_csv(r'C:\Users\danie\OneDrive\Documents\Repositories\oDCM-project-team-3\src\collection\huizenzoeker_province_data.csv') #at province-level

In [22]:
huizenzoeker_province = pd.read_csv('huizenzoeker_province_data1.csv', encoding= 'latin-1')

In [23]:
huizenzoeker_province = pd.DataFrame(huizenzoeker_province)
huizenzoeker_province

,Province,gem_vraagprijs,perc_ver_vraagprijs,verk_woningen,perc_ver_verkocht,gem_m2prijs,perc_ver_m2prijs,perc_overboden,perc_ver_overboden,best_inkomen
0,Noord-Holland,437000.0,16.53,812,-33.71,4508,10.76,12.66,1.08,36200
1,Zuid-Holland,365000.0,7.67,1094,-43.58,3630,6.17,10.21,0.77,35800
2,Zeeland,275000.0,0.09,145,-44.44,2663,3.62,8.13,0.13,36900
3,Noord-Brabant,365000.0,7.67,635,-50.20,3218,6.10,7.87,0.88,38100
4,Utrecht,439000.0,14.03,523,-17.77,4200,4.09,11.97,0.78,39500
5,Flevoland,340000.0,4.62,127,-46.41,2970,0.34,14.71,1.08,39500
6,Friesland,289000.0,3.58,286,-19.89,2446,-1.13,10.64,1.94,34900
7,Groningen,255000.0,13.33,282,-16.07,2592,8.41,15.76,1.33,30600
8,Drenthe,299750.0,1.61,208,-32.03,2475,0.69,10.49,1.41,37100
9,Overijssel,300000.0,0.67,392,-24.47,2732,5.65,9.84,1.38,36900


In [24]:
huizenzoeker_province.describe()

,gem_vraagprijs,perc_ver_vraagprijs,verk_woningen,perc_ver_verkocht,gem_m2prijs,perc_ver_m2prijs,perc_overboden,perc_ver_overboden,best_inkomen
count,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
mean,333645.833333,6.559167,454.750000,-34.221667,3086.416667,4.365833,10.975000,1.000000,36483.333333
std,61324.440695,5.593182,293.471735,12.065189,693.881108,3.441874,2.435407,0.478957,2394.627826
min,255000.000000,0.090000,127.000000,-50.200000,2446.000000,-1.130000,7.870000,0.130000,30600.000000
25%,286500.000000,1.570000,263.500000,-43.795000,2562.750000,1.987500,9.597500,0.760000,35575.000000
50%,320000.000000,6.040000,358.000000,-36.470000,2851.000000,4.680000,10.520000,0.980000,36900.000000
75%,365000.000000,9.085000,630.500000,-23.325000,3321.000000,6.117500,12.142500,1.342500,37650.000000
max,439000.000000,16.530000,1094.000000,-16.070000,4508.000000,10.760000,15.760000,1.940000,39500.000000
